In [2]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, ScalarFormatter

## Data Upload

In [3]:
# Загружуем таблицы из RDF commander по какому либо месторождению 
mer_init = pd.read_csv('kar/kar_mer.csv', encoding = 'utf8', sep = ';')
rigis_init = pd.read_csv('kar/kar_rigis.csv', encoding = 'utf8', sep = ';')
coord_init = pd.read_csv('kar/kar_xyzones.csv', encoding = 'utf8', sep = ';')

In [4]:
# Ниже парсится выгрузка МЭР из RDF commander по какому либо месторождению 
oilfieldSplit = pd.Series(mer_init.WELL).str.split('@')
oilfieldSplit_inter = pd.DataFrame(oilfieldSplit.to_list(), columns = ['oilfield', 'col1', 'col2', 'well'])
oilfieldSplit_inter = oilfieldSplit_inter[['oilfield','well']]
mer_oilfield = mer_init.join(oilfieldSplit_inter)
mer_oilfield.head()

,WELL,Date,kPlast,kCharWork,kState,kMethod,WorkTime,CollTime,StayTime,kStayReason,...,kAgentInj,OilV,WatV,OtherFluidV,Injection,WaterLoss,TechInj,Unnamed: 24,oilfield,well
0,Карамовское@all@skv@100,01.11.1984,БС11,НЕФ,РАБ.,ФОН,24,0,0,NaN,...,NaN,12.0,0.0,0.0,0.0,0.0,0.0,NaN,Карамовское,100
1,Карамовское@all@skv@100,01.12.1984,БС11,НЕФ,РАБ.,ФОН,744,0,0,NaN,...,NaN,346.0,0.0,0.0,0.0,0.0,0.0,NaN,Карамовское,100
2,Карамовское@all@skv@100,01.01.1985,БС11,НЕФ,РАБ.,ФОН,744,0,0,NaN,...,NaN,313.0,0.0,0.0,0.0,0.0,0.0,NaN,Карамовское,100
3,Карамовское@all@skv@100,01.02.1985,БС11,НЕФ,РАБ.,ФОН,672,0,0,NaN,...,NaN,805.0,0.0,0.0,0.0,0.0,0.0,NaN,Карамовское,100
4,Карамовское@all@skv@100,01.03.1985,БС11,НЕФ,РАБ.,ФОН,744,0,0,NaN,...,NaN,747.0,0.0,0.0,0.0,0.0,0.0,NaN,Карамовское,100


In [5]:
# Ниже переименование и определение типов данных таблицы МЭР 
mer_oilfield.rename(columns = 
            {'Date':'date',
            'kPlast':'zone',
            'kCharWork':'type_work',
            'kState':'state',
            'kMethod':'method',
            'WorkTime':'work',
            'CollTime':'coll',
            'StayTime':'stay',
            'kStayReason':'stay_reason',
            'Oil':'oil',
            'Wat':'wat',
            'Gas':'gas',
            'NaturalGas':'natur_gas',
            'GasCondensate':'gas_condens',
            'CapGas':'cap_gas',
            'kAgentProd':'agent_prod',
            'kAgentInj':'ageny_inj',
            'OilV':'oil_v',
            'WatV':'wat_v',
            'OtherFluidV':'fluid_v',
            'Injection':'inj',
            'WaterLoss':'wat_loss',
            'TechInj':'tech_inj',}, inplace=True)
mer = mer_oilfield[[ 'oilfield','well','date','zone','type_work','state','method','work','coll','stay','stay_reason','oil','wat','gas','oil_v',
           'wat_v','fluid_v','inj','wat_loss','tech_inj']]
mer['date'] = pd.to_datetime(mer['date'], dayfirst=True)
convert_dict = {'oilfield': 'string',
                'well': 'string',
                'zone': 'string',
                'type_work': 'string',
                'state': 'string',
                'method': 'string',
                'work': 'float64',
                'coll': 'float64',
                'stay': 'float64',
                'stay_reason': 'string',
               } 
mer = mer.astype(convert_dict)
mer['oil_tpd'] = (mer.oil/mer.work)*24
mer['wat_tpd'] = (mer.wat/mer.work)*24
mer['gas_upd'] = (mer.gas/mer.work)*24
mer['wc'] = mer.wat_tpd/(mer.oil_tpd + mer.wat_tpd)

In [6]:
mer.head()

,oilfield,well,date,zone,type_work,state,method,work,coll,stay,...,oil_v,wat_v,fluid_v,inj,wat_loss,tech_inj,oil_tpd,wat_tpd,gas_upd,wc
0,Карамовское,100,1984-11-01,БС11,НЕФ,РАБ.,ФОН,24.0,0.0,0.0,...,12.0,0.0,0.0,0.0,0.0,0.0,10.000000,0.0,0.000000,0.0
1,Карамовское,100,1984-12-01,БС11,НЕФ,РАБ.,ФОН,744.0,0.0,0.0,...,346.0,0.0,0.0,0.0,0.0,0.0,9.645161,0.0,482.677419,0.0
2,Карамовское,100,1985-01-01,БС11,НЕФ,РАБ.,ФОН,744.0,0.0,0.0,...,313.0,0.0,0.0,0.0,0.0,0.0,8.741935,0.0,468.806452,0.0
3,Карамовское,100,1985-02-01,БС11,НЕФ,РАБ.,ФОН,672.0,0.0,0.0,...,805.0,0.0,0.0,0.0,0.0,0.0,24.857143,0.0,1248.035714,0.0
4,Карамовское,100,1985-03-01,БС11,НЕФ,РАБ.,ФОН,744.0,0.0,0.0,...,747.0,0.0,0.0,0.0,0.0,0.0,20.838710,0.0,1047.354839,0.0


In [7]:
rigis_init.rename(columns = 
            {'Oilfield':'oilfield',
            'Well/Wellbore':'well',
            'plast':'zone',
            'plastzone':'zone_z',
            'H':'md',
            'L':'h_md',
            'Habs':'tvdss',
            'Labs':'h_tvdss', 
            'Lit':'lit',
            'Collector':'collector',
            'Sat':'satur',
            'Ro':'rt',
            'APS':'aps',
            'aGK':'agk',
            'Kpr':'kpr',
            'Kpor':'kp',
            'Kn':'kng',
            'Kgl':'kgl'}, inplace=True)
rigis = rigis_init[['oilfield','well','zone','md','h_md','tvdss','h_tvdss','lit','collector','aps','agk','rt','kgl','kp','kpr','kng','satur']]
convert_dict = {'oilfield': 'string',
                'well': 'string',
                'zone': 'string',
                'lit': 'string',
                'agk': 'float64',
                'satur': 'string',
               } 
rigis = rigis.astype(convert_dict)

In [8]:
rigis.head()

,oilfield,well,zone,md,h_md,tvdss,h_tvdss,lit,collector,aps,agk,rt,kgl,kp,kpr,kng,satur
0,Карамовское,100,БС8-1,2659.4,0.1,2412.9,0.1,НЯ,True,0.87,0.0,4.9,0.0,18.5,0.0,0.0,В
1,Карамовское,100,БС8-1,2659.5,2.7,2413.0,2.7,П,True,0.84,0.0,5.4,9.3,21.3,73.0,14.7,В
2,Карамовское,100,БС8-1,2662.2,0.9,2415.7,0.9,Г,False,0.18,0.0,0.0,41.6,12.3,0.1,0.0,<NA>
3,Карамовское,100,БС8-1,2663.1,1.0,2416.6,1.0,П,True,0.60,0.0,0.0,9.3,19.6,7.2,0.0,В
4,Карамовское,100,БС8-1,2664.1,1.5,2417.6,1.5,Г,False,0.22,0.0,0.0,36.7,12.2,0.1,0.0,<NA>


In [9]:
coord_init.rename(columns = 
            {'Oilfield':'oilfield',
            'Pl':'zone',
            'Well/Wellbore':'well',
            'X':'x',
            'Y':'y',
            'X0':'x0',
            'Y0':'y0',}, inplace=True)
coord = coord_init[[ 'oilfield','well','zone','x','y','x0','y0']]
convert_dict = {'oilfield': 'string',
                'well': 'string',
                'zone': 'string',
                'x': 'float64',
                'y': 'float64',
                'x0': 'float64',
                'y0': 'float64'} 
coord = coord.astype(convert_dict)
# делаем целыми и текстовыми координаты устья скважин
int_dict = {'x0': 'int',
            'y0': 'int',
           }
coord = coord.astype(int_dict)
str_dict = {'x0': 'string',
            'y0': 'string',
           }
coord = coord.astype(str_dict)
# coord.head()
# собираем скважины в списки по координатам устья
coord['x0y0'] = coord[['x0','y0']].agg(' '.join, axis=1)
coord_wh = coord[['well','x0y0']]
coord_wh = pd.DataFrame(coord.groupby('well')['x0y0'].max())
coord_wh.reset_index(inplace=True)
coord_wells = pd.DataFrame(coord_wh.groupby('x0y0')['well'].apply(lambda x: np.array(x)))
coord_wells.reset_index(inplace=True)
# coord_wells.head()

In [10]:
coord.head()

,oilfield,well,zone,x,y,x0,y0,x0y0
0,Карамовское,100,БС11,479391.0,7006795.0,478943,7006422,478943 7006422
1,Карамовское,100,БС10-3,479391.0,7006795.0,478943,7006422,478943 7006422
2,Карамовское,100,БС10-2,479391.0,7006795.0,478943,7006422,478943 7006422
3,Карамовское,100,БС10-1,479391.0,7006795.0,478943,7006422,478943 7006422
4,Карамовское,100,БС9,479391.0,7006795.0,478943,7006422,478943 7006422


## Data Processing

## Final Result